## America's Working Class and How It Helped Swing the 2016 Election

Many pundits have argued that this past election was decided by America's working class. It has always been the case that jobs and the economy play a pivotal role in influencing the voting decisions of many Americans. With the Great Recession of 2008-2011, and an increasing shift towards a globalized economy, traditionally blue-collar industries such as construction, manufacturing, and transportation and trade felt the brunt of those effects through the loss of jobs. 

We hypothesize in this project that the people most affected by these changes - America's working class - responded to these changes with their votes, and may correlate with whether a state went red or blue in 2016. While obviously it's hard to definitively pin down a cause just based on how people voted, we want to compare labor department data about umeployment in certain sectors and voting data from certain states to see if we can find a trend or come to any interesting conclusions.

For our project, we look towards a number of sources to aid in our analysis. First, on the labor statistics front, we looked towards the [Bureau of Labor Statistics](http://www.bls.gov/bls/proghome.htm) and, specifically, the [Bureau of Labor Statistics Public Data API](http://www.bls.gov/developers/api_signature_v2.htm) to collect data on unemployment over the industries most impacted by the recession, and break down this data by state. We will combine this feature with the recovery rate of a given state by 2016. Second, for election data, we turn towards major news sources like [Politico](http://www.politico.com/) and the [New York Times](http://www.nytimes.com) to collect voter numbers from each state during the 2008, 2012, and 2016 elections. 

## Analyzing the Election

Though both Politico and the New York Times, as many news sources do, ultimately draw their data from the Associated Press, API keys for AP aren't free, so we'll have to take a slightly more indirect approach by scraping the news pages themselves for the data instead of getting it directly from AP. Unfortunately, due to formatting changes over the years, it's a little more difficult than we'd like it to be to fetch voter data from the 2008, 2012, and 2016 elections with one script. However, it's not too hard to write scripts for each of those elections individually.

In [49]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# pd.options.display.max_seq_items = 2000
# pd.set_option('display.height', 1000)
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

In [50]:
import pandas as pd
import numpy as np
import requests
import sklearn
import time
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from selenium import webdriver

years = ['2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']


# List of states and D.C.
states = [
    'Alabama','Alaska','Arizona','Arkansas','California','Colorado',
    'Connecticut','Delaware','District Of Columbia','Florida','Georgia','Hawaii','Idaho', 
    'Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana',
    'Maine','Maryland','Massachusetts','Michigan','Minnesota',
    'Mississippi', 'Missouri','Montana','Nebraska','Nevada',
    'New Hampshire','New Jersey','New Mexico','New York',
    'North Carolina','North Dakota','Ohio',    
    'Oklahoma','Oregon','Pennsylvania','Rhode Island',
    'South Carolina','South Dakota','Tennessee','Texas','Utah',
    'Vermont','Virginia','Washington','West Virginia',
    'Wisconsin','Wyoming'
]

state_codes = {
    'Alabama' : '01',
    'Alaska' : '02',
    'Arizona' : '04',
    'Arkansas' : '05',
    'California' : '06',
    'Colorado' : '08',
    'Connecticut' : '09',
    'Delaware' : '10',
    'District Of Columbia' : '11',
    'Florida' : '12',
    'Georgia' : '13',
    'Hawaii' : '15',
    'Idaho' : '16', 
    'Illinois' : '17',
    'Indiana' : '18',
    'Iowa' : '19',
    'Kansas' : '20',
    'Kentucky' : '21',
    'Louisiana' : '22',
    'Maine' : '23',
    'Maryland' : '24',
    'Massachusetts' : '25',
    'Michigan' : '26',
    'Minnesota' : '27',
    'Mississippi' : '28',
    'Missouri' : '29',
    'Montana' : '30',
    'Nebraska' : '31',
    'Nevada' : '32',
    'New Hampshire' : '33',
    'New Jersey' : '34',
    'New Mexico' : '35',
    'New York' : '36',
    'North Carolina' : '37',
    'North Dakota' : '38',
    'Ohio' : '39',    
    'Oklahoma' : '40',
    'Oregon' : '41',
    'Pennsylvania' : '42',
    'Rhode Island' : '44',
    'South Carolina' : '45',
    'South Dakota' : '46',
    'Tennessee' : '47',
    'Texas' : '48',
    'Utah' : '49',
    'Vermont' : '50',
    'Virginia' : '51',
    'Washington' : '53',
    'West Virginia' : '54',
    'Wisconsin' : '55',
    'Wyoming' : '56'
    }

industries = ['information', 'finance', 'construction', 'manufacturing', 'transport_and_trade', 'all']

First, we'll be collecting 2008 data from the New York Times page on the election. In terms of formatting, we're hoping to get mainly the state and the number of votes for the Democratic and Republican party candidates (we'll be ignoring third-parties in this analysis).

In [51]:
'''
Converts data taken from the table in the html
    Input: a table row of voting data, in html
    Output: a list of data in a more desirable format
'''
def voter_html_to_data_2008(voter_html):
    data = [datum.get_text().strip() for datum in voter_html.find_all('td')]
    blue_votes = int(data[2][:-5].replace(',',''))
    red_votes = int(data[4][:-5].replace(',',''))
    return [blue_votes, red_votes]

def get_voter_data_2008():
    # Set up our data frame
    df = pd.DataFrame(columns=('election_year', 'state', 'dem_votes', 'rep_votes'))
    
    # Base url we'll be getting data from 
    base_url = 'http://elections.nytimes.com/2008/results/states/president/'
    
    # Get state names for url endings
    state_urls = sorted([state.lower().replace(' ', '-') for state in states])

    # Iterate through the states (except for Alaska and D.C.)
    num_states = 0
    for state in state_urls:
        if state != 'alaska' and state != 'district-of-columbia':
            # Get data from the site
            response = requests.get(base_url + state + '.html')
            election_soup = BeautifulSoup(response.text, 'html.parser')

            # Data for all states except for Alaska and D.C.
            data_rows = election_soup.find(id='winners-by-county-table').tbody.find_all('tr')
            
            # Data that every row will have (election year and state)
            header_data = ['2008', state]
            state_vote_lists = [voter_html_to_data_2008(row) for row in data_rows]
            state_vote_counts = [sum([vote[0] for vote in state_vote_lists]), sum([vote[1] for vote in state_vote_lists])]
            voter_data = header_data + state_vote_counts
            df.loc[num_states] = voter_data
            num_states += 1
    
    # Since Alaska and D.C. don't have counties, we process them slightly differently
    # First, Alaska
    alaska_html = requests.get('http://elections.nytimes.com/2008/results/states/alaska.html')
    alaska_election_soup = BeautifulSoup(alaska_html.text, 'html.parser')
    
    alaska_obama = alaska_election_soup.find(id='presidential-results-table').tbody.find_all('tr')[:2][1].find_all('td')
    alaska_mccain = alaska_election_soup.find(id='presidential-results-table').tbody.find_all('tr')[:2][0].find_all('td')
    alaska_blue_votes = int(alaska_obama[1].get_text().strip().replace(',',''))
    alaska_red_votes = int(alaska_mccain[2].get_text().strip().replace(',',''))
    alaska_data = ['2008', 'alaska', alaska_blue_votes, alaska_red_votes]
    df.loc[num_states] = alaska_data
    num_states += 1
    
    # Finally, D.C.
    dc_html = requests.get('http://elections.nytimes.com/2008/results/states/district-of-columbia.html')
    dc_election_soup = BeautifulSoup(dc_html.text, 'html.parser')

    dc_obama = dc_election_soup.find(id='presidential-results-table').tbody.find_all('tr')[:2][0].find_all('td')
    dc_mccain = dc_election_soup.find(id='presidential-results-table').tbody.find_all('tr')[:2][1].find_all('td')
    dc_blue_votes = int(dc_obama[2].get_text().strip().replace(',',''))
    dc_red_votes = int(dc_mccain[1].get_text().strip().replace(',',''))
    dc_data = ['2008', 'district-of-columbia', dc_blue_votes, dc_red_votes]
    df.loc[num_states] = dc_data
    num_states += 1
    
    return df

In [52]:
df_2008 = get_voter_data_2008()

In [53]:
df_2008

,election_year,state,dem_votes,rep_votes
0,2008,alabama,811764.0,1264879.0
1,2008,arizona,948648.0,1132560.0
2,2008,arkansas,418049.0,632672.0
3,2008,california,7441458.0,4554643.0
4,2008,colorado,1216793.0,1020135.0
5,2008,connecticut,994320.0,627688.0
6,2008,delaware,255394.0,152356.0
7,2008,florida,4143957.0,3939380.0
8,2008,georgia,1843452.0,2048244.0
9,2008,hawaii,324918.0,120309.0


For 2012, we'll switch to Politico, since the website formatting is slightly easier to parse than the interactive graphics on the New York Times site.

In [54]:
def get_voter_data_2012():
    # Set up our data frame
    df = pd.DataFrame(columns=('election_year', 'state', 'dem_votes', 'rep_votes'))
    
    # Base url we'll be getting data from
    base_url = 'http://www.politico.com/2012-election/results/president/'
    
    # Get state names for url endings
    state_urls = sorted([state.lower().replace(' ', '-') for state in states])
    
    # Once again, Alaska and D.C. are slightly different
    num_states = 0
    for state in state_urls:
        # Get data from site
        try:
            response = requests.get(base_url + state + '/')
        except ConnectionError:
            time.sleep(2)
            response = requests.get(base_url + state + '/')
        election_soup = BeautifulSoup(response.text, 'html.parser')
        header_data = ['2012', state]
        data = election_soup.find('div', class_='state-results-macro').table.tbody
        blue_votes = int(data.find(class_='party-democrat').find(class_='results-popular').get_text().strip().replace(',',''))
        red_votes = int(data.find(class_='party-republican').find(class_='results-popular').get_text().strip().replace(',',''))
        voter_data = header_data + [blue_votes, red_votes]
        df.loc[num_states] = voter_data
        num_states += 1
        
    return df

In [55]:
df_2012 = get_voter_data_2012()

In [56]:
df_2012

,election_year,state,dem_votes,rep_votes
0,2012,alabama,793620.0,1252453.0
1,2012,alaska,102138.0,136848.0
2,2012,arizona,930669.0,1143051.0
3,2012,arkansas,389699.0,638467.0
4,2012,california,6493924.0,4202127.0
5,2012,colorado,1238490.0,1125391.0
6,2012,connecticut,912531.0,631432.0
7,2012,delaware,242547.0,165476.0
8,2012,district-of-columbia,222332.0,17337.0
9,2012,florida,4235270.0,4162081.0


Finally, for 2016, we'll be using Politico again.

In [57]:
def voter_html_to_data_2016(voter_html):
    blue_votes = int(voter_html.find(class_='type-democrat').find(class_='results-popular').get_text().replace(',',''))
    red_votes = int(voter_html.find(class_='type-republican').find(class_='results-popular').get_text().replace(',',''))
    return [blue_votes, red_votes]

def get_voter_data_2016():
    # Set up our data frame
    df = pd.DataFrame(columns=('election_year', 'state', 'dem_votes', 'rep_votes'))
    
    # Base url we'll be getting data from
    base_url = 'http://www.politico.com/2016-election/results/map/president/'
    
    # Get state names for url endings
    state_urls = sorted([state.lower().replace(' ', '-') for state in states])
    
    # This time, Alaska and D.C. aren't different!
    num_states = 0
    for state in state_urls:
        # Get data from site
        response = requests.get(base_url + state + '/')
        election_soup = BeautifulSoup(response.text, 'html.parser')

        data = election_soup.select('section.content-group.election-intro')[0].find('div', class_='overall')
        header_data = ['2016', state]
        voter_data = header_data + voter_html_to_data_2016(data)
        df.loc[num_states] = voter_data
        num_states += 1
    
    return df

In [58]:
df_2016 = get_voter_data_2016()

In [59]:
df_2016

,election_year,state,dem_votes,rep_votes
0,2016,alabama,718084.0,1306925.0
1,2016,alaska,93007.0,130415.0
2,2016,arizona,936250.0,1021154.0
3,2016,arkansas,378729.0,677904.0
4,2016,california,7362490.0,3916209.0
5,2016,colorado,1208095.0,1136354.0
6,2016,connecticut,884432.0,668266.0
7,2016,delaware,235581.0,185103.0
8,2016,district-of-columbia,260223.0,11553.0
9,2016,florida,4485745.0,4605515.0


To aid in our analysis, we'll create a simple function to find the percent of the state that voted blue, given an election year.

In [60]:
def find_percent_blue(state, year):
    if year == 2008:
        df = df_2008
    elif year == 2012:
        df = df_2012
    else:
        df = df_2016
    df = df.set_index('state')
    dem_votes = df.at[state,'dem_votes']
    rep_votes = df.at[state, 'rep_votes']
    return dem_votes / (dem_votes + rep_votes)

In [61]:
print find_percent_blue('california', 2012)
print find_percent_blue('california', 2016)

0.60713285679
0.652778303597


Next, we'll begin gathering data from the BLS, specifically, data regarding employment by state in the industries most negatively impacted by the recession (construction, manufacturing, and transportation/trade), as well as overall employment, in thousands of employees. 

---

## Scraping Labor Statistics

In [62]:
import json, string
import requests, tqdm
from collections import defaultdict

To utilize the Bureau of Labor Statistics Public API is unintuitive, to say the least. Essentially, each unique dataset that the BLS keeps has a serial ID that correesponds to it. Querying the data from the API involves figuring out exactly which dataset you want, and then translating that to a serial ID. The code below allows us to input requests in a reasonable format like JSON and have that get converted to the serial ID that we need.

In [87]:
def opand(x,y): 
    return x and y


def nest(keys, d_keys, d):
    assert (len(keys) == len(d_keys))
    iter_format = lambda (i, d_key) : "for x%d in d['%s']" % (i, d_key)
    iters = string.join(map(iter_format, enumerate(d_keys)), sep=" ")
    arg_format = lambda (i) : '("' + keys[i] + '", x' + str(i) + ')'
    args = string.join(map(arg_format, range(len(keys))), sep=", ")
    return eval('[dict([' +  args +  ']) ' + iters + ']')


def request(series):
    """
    brief:
        - requests series in batches of 15, returns list of json_data, 
          one for each request
    args:
        - series : list of bls series id's
    """
    bls_url = 'http://api.bls.gov/publicAPI/v2/timeseries/data/'
    regKeys = ["bc2b9775e9794f37a23c0f6b2a4659b1", 
               "8874280f2cc142f3833754ad3cb6855c",
               "dacaff7ea67b466685622348e9807f8f"]
    json_data = []
    for i in tqdm.tqdm(range(0,len(series),50)):
        headers = {'Content-type': 'application/json'}
        data = json.dumps({"seriesid": series[i:i+50],
                           "startyear":"2004", 
                           "endyear":"2016", 
                           "registrationKey":regKeys[2]})
        try:
            response = requests.post(bls_url, data=data, headers=headers)
        except ConnectionError:
            time.sleep(2)
            response = requests.post(bls_url, data=data, headers=headers)
        json_data.append(json.loads(response.text))
    return json_data


def format_series(config):
    """
    brief:
        - formats a request for data from the bls state and area employment, 
          hours, and earnings database if config['prefix'] == 'SM'

        - formats a request for data from the bls local area unemployment 
          statistics database if config['prefix'] == 'SA'
    args:
        - config : {                   # http://www.bls.gov/help/hlpforma.htm#SM
                    'prefix' : 'SM',
                    'seasonal_adjustment' : 'S' or 'U',
                    'state_code' : '01' to '50', 
                    'area_code' : '00000' to '99999',
                    'supersector' : '00' to '99', 
                    'industry' : '000000' to '999999',     
                    'datatype' : '00' to '99'              
                    }
                or {                   # http://www.bls.gov/help/hlpforma.htm#LA
                    'prefix' : 'LA', 
                    'seasonal_adjustment' : 'S' or 'U',
                    'area_type' : 'ST' or 'MT', 
                    'state_code' : '01' to '50',
                    'area_code' : '000000000' or 'YYYYYYYYYYY'
                    'measure': '03' to '06'
                   }
    """
    keys = dict({'SM' : ['prefix','seasonal_adjustment','state_code','area_code','supersector','industry','datatype'],
                 'LA' : ['prefix', 'seasonal_adjustment','state_area_code', 'measure'],
                 })
    prefix = config['prefix']
    return string.join([config[key] for key in keys[prefix]], sep="")


def make_series(config):
    """
    brief:
        - provided a dictionary of bls series id parameters,
          construct and request each possible series_id given parameters
    args:
        - config : {
                    'prefix' : ['SM'],
                    'seasonal_adjustments' : ['S' or 'U'],
                    'state_codes' : [string],
                    'area_codes' : [{string -> string}],
                    'supersectors' : [string],
                    'industries' : [string],
                    'datatypes' : [string]
                    }
                or {
                    'prefix' : ['LA'],
                    'seasonal_adjustment' : ['S' or 'U'],
                    'area_types' : ['ST' or 'MT'],
                    'state_codes' : ['01' to '50']
                    'area_codes' : [{'ST', 'xx' --> 'STxx00000000000', 
                                    'MT', 'xx' --> 'MTxxYYYYYYYYYYY'}]
                    'measures': [string]
                   }
    """
    keys = dict({'SM' : ['prefix','seasonal_adjustment','state_code','area_code','supersector','industry','datatype'],
                 'LA' : ['prefix','seasonal_adjustment','state_area_code','measure'],
                 })
    param_keys = dict({'SM' : ['prefix','seasonal_adjustments','state_codes','area_codes','supersectors','industries','datatypes'],
                       'LA' : ['prefix','seasonal_adjustments','state_area_codes','measures'],
                       })
    prefix = config['prefix'][0]
    return [format_series(sid) for sid in nest(keys[prefix], param_keys[prefix], config)]

With that, now we can actually use the API to get some useful information. Here, with a provided listed of metro codes (BLS codes corresponding to different metropolitan areas in the U.S.), and using those, we can construct queries to fetch various employment statistics from different states and metropolitan areas.

In [88]:
config = {'prefix' : ['SM'],
          'seasonal_adjustments' : ['S', 'U'],
          'state_codes' : state_codes.values(),
          'area_codes' : ['00000'],
          'supersectors' : ['20', '30', '40', '50', '55', '00'],
          'industries' : ['000000'],
          'datatypes' : ['01'],
          }
           
json_data_list = request(make_series(config))


  0%|          | 0/13 [00:00<?, ?it/s]

<Response [405]>


ValueError: No JSON object could be decoded

Now that we've retrieved our data from the BLS API, we'll format it into the Pandas dataframes we'll be performing analyses on in the next section.

In [ ]:
def series_id_to_state(series_id):
    state_code_to_string = dict(zip(state_codes.values(), state_codes.keys()))
    return state_code_to_string[str(series_id[3:5])]

def series_id_to_industry(series_id):
    industry_code_to_string = dict({'50' : 'information',
                                    '55' : 'finance',
                                    '20' : 'construction',
                                    '30' : 'manufacturing',
                                    '40' : 'transport_and_trade',
                                    '00' : 'all',
                                   })
    return industry_code_to_string[series_id[10:12]]


industry_data = dict({'information' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      'finance' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      'construction' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      'manufacturing' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      'transport_and_trade' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      'all' : dict(zip(states, [defaultdict(list) for i in range(len(states))])),
                      })

# each json_data request has at between 1 <= # requests <= 50 in a list
for json_data in json_data_list:
    # each series in this list spans the years 2004:2016, and months 01:12
    for json_series in json_data['Results']['series']:
        series_id = json_series['seriesID']
        # the state, and industry is constant for each series
        state = series_id_to_state(series_id)
        industry = series_id_to_industry(series_id)
        # we want to average the values for each month for a given year
        for year in years:
            year_avg = []
            for entry in json_series['data']:
                if entry['year'] == year:
                    year_avg.append(float(entry['value']))
            year_avg = (0 if year_avg == [] else sum(year_avg) / len(year_avg))
            industry_data[industry][state][year].append(year_avg)

industry_dfs = dict()
for industry in industries:
    state_data = industry_data[industry]
    for state in states:
        year_data = state_data[state]
        coalesce = lambda year : sum(year_data[year]) / len(year_data[year]) if year_data[year] != [] else 0
        state_data[state] = [coalesce(year) for year in years]
    industry_dfs[industry] = pd.DataFrame(data=state_data, index=years, columns=states).transpose()

In [ ]:
construction_df = industry_dfs['construction']
manufacturing_df = industry_dfs['manufacturing']
transport_and_trade_df = industry_dfs['transport_and_trade']
total_employment_df = industry_dfs['all']

In [ ]:
construction_df
manufacturing_df
transport_and_trade_df
total_employment_df

Now that all of our data is properly formatted, we can perform our analysis. In the next section, we manipulate the dataframes we created with the goal of predicting whether a state will be red or blue in 2016. We do this by training an SVC on the voter and employment datasets for 2008 and 2012 that we generated.

## Analyzing Employment Data

First, we aggregate the employment of our blue collar industries into one big dataframe listing each state's blue collar employees in each election year.

In [ ]:
def create_blue_collar_df(construction, manufacturing, transport_trade):
    df = pd.DataFrame({
            "state": states,
            2004: construction['2004'] + manufacturing['2004'] + transport_trade['2004'],
            2008: construction['2008'] + manufacturing['2008'] + transport_trade['2008'],
            2012: construction['2012'] + manufacturing['2012'] + transport_trade['2012'],
            2016: construction['2016'] + manufacturing['2016'] + transport_trade['2016']
        })
    return df
    
blue_collar_df = create_blue_collar_df(construction_df, manufacturing_df, transport_and_trade_df)

We then use this dataframe and the total employment dataframe to make our SVM input dataframes.

In [ ]:
def create_SVM_input(year, blue_collar_df, total_employment_df, df_4_years_ago):
    df = pd.DataFrame({"state": states, 
                       "blue_collar_change_last_8_years": (blue_collar_df[year] - blue_collar_df[year - 8]) / blue_collar_df[year - 8],
                       "total_employment_change_last_8_years": (total_employment_df[year] - total_employment_df[year - 8]) / total_employment_df[year - 8]})
    df["proportion_blue_4_years_ago"] = find_percent_blue(df["state"], year)
    return df

df_predict_2012 = create_SVM_input(2012, blue_collar_df, total_employment_df, df_2008)
df_predict_2016 = create_SVM_input(2016, blue_collar_df, total_employment_df, df_2012)

Next we label each state with its result (1 for Democrat, 0 for Republican) in each of the last 3 presidential elections (2008, 2012, 2016).

In [ ]:
results = {
    'Alabama': [0,0,0], 'Alaska': [0,0,0],
    'Arizona': [0,0,0], 'Arkansas': [0,0,0],
    'California': [1,1,1], 'Colorado': [1,1,1],
    'Connecticut': [1,1,1], 'Delaware': [1,1,1],
    'District Of Columbia': [1,1,1], 'Florida': [1,1,0],
    'Georgia': [0,0,0], 'Hawaii': [1,1,1],
    'Idaho': [0,0,0], 'Illinois': [1,1,1],
    'Indiana': [1,0,0], 'Iowa': [1,1,0],
    'Kansas': [0,0,0], 'Kentucky': [0,0,0],
    'Louisiana': [0,0,0], 'Maine': [1,1,1],
    'Maryland': [1,1,1],'Massachusetts': [1,1,1],
    'Michigan': [1,1,0],'Minnesota': [1,1,1],
    'Mississippi': [0,0,0], 'Missouri': [0,0,0],
    'Montana': [0,0,0], 'Nebraska': [0,0,0],
    'Nevada': [1,1,1], 'New Hampshire': [1,1,1],
    'New Jersey': [1,1,1],'New Mexico': [1,1,1],
    'New York': [1,1,1], 'North Carolina': [1,0,0],
    'North Dakota': [0,0,0],'Ohio': [1,1,0],    
    'Oklahoma': [0,0,0],'Oregon': [1,1,1],
    'Pennsylvania': [1,1,0],'Rhode Island': [1,1,1],
    'South Carolina': [0,0,0],'South Dakota': [0,0,0],
    'Tennessee': [0,0,0],'Texas': [0,0,0],
    'Utah': [0,0,0], 'Vermont': [1,1,1],
    'Virginia': [1,1,1],'Washington': [1,1,1],
    'West Virginia': [0,0,0], 'Wisconsin': [1,1,0],
    'Wyoming': [0,0,0]
}

def provide_labels(year):
    return np.array([v[(year - 2008) / 4] for k, v in results.iteritems()])
    
# df_predict_2012["alphabetical_state_results_2008"] = provide_labels(2008)
df_predict_2012["alphabetical_state_results_2012"] = provide_labels(2012)
df_predict_2016["alphabetical_state_results_2016"] = provide_labels(2016)

Now that we've extracted all the necessary training data, we train an SVM on the features and labels of our training set.

In [ ]:
# classifier for training examples, which are 2012 states and labels
def learn_classifier(training_features, training_labels):
    svc = sklearn.svm.SVC()
    return svc.fit(training_features, training_labels)

trained_svc_industry_to_total = learn_classifier(df_predict_2012["blue_collar_change_last_8_years", 
                                               "total_employment_change_last_8_years"], 
                               df_predict_2012["alphabetical_state_results_2012"])

trained_svc_industry_to_vote = learn_classifier(df_predict_2012["blue_collar_change_last_8_years",
                                               "proportion_blue_4_years_ago"], 
                               df_predict_2012["alphabetical_state_results_2012"])

trained_svc_total_to_vote = learn_classifier(df_predict_2012["total_employment_change_last_8_years", 
                                               "proportion_blue_4_years_ago"], 
                               df_predict_2012["alphabetical_state_results_2012"])

Finally, we run our classifier on our testing set, the states in the 2016 election.

In [ ]:
def eval_classifier(svc, testing_features, testing_labels):
    predicted_labels = svc.predict(testing_features)
    return float(np.sum(predicted_labels == testing_labels)) / testing_features.shape[0]

accuracy_industry_to_total = eval_classifier(trained_svc_industry_to_total, df_predict_2016["blue_collar_change_last_8_years", 
                                                        "total_employment_change_last_8_years"], 
                           df["alphabetical_state_results_2016"])

accuracy_industry_to_vote = eval_classifier(trained_svc_industry_to_vote, df_predict_2016["blue_collar_change_last_8_years",
                                                        "proportion_blue_4_years_ago"], 
                           df["alphabetical_state_results_2016"])

accuracy_total_to_vote = eval_classifier(trained_svc_total_to_vote, df_predict_2016["total_employment_change_last_8_years", 
                                                        "proportion_blue_4_years_ago"], 
                           df["alphabetical_state_results_2016"])


print accuracy_industry_to_total, accuracy_industry_to_vote, accuracy_total_to_vote